In [42]:
import pandas as pd
from textblob import TextBlob
from textblob import classifiers

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import precision_recall_fscore_support as score
import sklearn


In [4]:
df = pd.read_csv('clean_dataset.csv', sep = ';', encoding='latin-1')
df.head()

,productid,userid,score,text,clean_text,sentiment
0,B001E4KFG0,A3SGXH7AUHU8GW,5,I have bought several of the Vitality canned d...,i bought several vitality canned dog food prod...,positive
1,B00813GRG4,A1D87F6ZCVE5NK,1,Product arrived labeled as Jumbo Salted Peanut...,product arrived labeled jumbo salted peanuts ....,negative
2,B000LQOCH0,ABXLMWJIXXAIN,4,This is a confection that has been around a fe...,this confection around centuries it light pill...,positive
3,B000UA0QIQ,A395BORC6FGVXV,2,If you are looking for the secret ingredient i...,if looking secret ingredient robitussin i beli...,negative
4,B006K2ZZ7K,A1UQRSCLF8GW1T,5,Great taffy at a great price. There was a wid...,great taffy great price there wide assortment ...,positive


## NaiveBayesClassifier with textblob

In [15]:
train_df = pd.DataFrame()
for productid_action in df['productid'].unique():
    productid_df = df.loc[df['productid'] == productid_action]
    n_instance = productid_df.shape[0]
    train_df = train_df.append(productid_df.sample(n = int(0.8 * n_instance), replace = False))

test_df = pd.concat([df, train_df]).drop_duplicates(keep=False)


In [17]:
train_df.shape

(25685, 6)

In [18]:
test_df.shape

(9465, 6)

In [19]:
train_df.to_csv('train_dataset.csv', sep = ';', index=False)
test_df.to_csv('test_dataset.csv', sep = ';', index=False)

In [20]:
training =[]
for index, rows in df.iterrows(): 
    # Create list for the current row 
    my_list =[rows.clean_text, rows.sentiment] 
      
    # append the list to the final list 
    training.append(my_list)

#Print the list 
training

[['i bought several vitality canned dog food products found good quality the product looks like stew processed meat smells better my labrador finicky appreciates product better',
  'positive'],
 ['product arrived labeled jumbo salted peanuts ... peanuts actually small sized unsalted not sure error vendor intended represent product jumbo',
  'negative'],
 ['this confection around centuries it light pillowy citrus gelatin nuts case filberts and cut tiny squares liberally coated powdered sugar and tiny mouthful heaven not chewy flavorful i highly recommend yummy treat if familiar story c s lewis the lion the witch the wardrobe treat seduces edmund selling brother sisters witch',
  'positive'],
 ['if looking secret ingredient robitussin i believe i found i got addition root beer extract i ordered good made cherry soda the flavor medicinal',
  'negative'],
 ['great taffy great price there wide assortment yummy taffy delivery quick if taffy lover deal',
  'positive'],
 ['i got wild hair taff

In [ ]:
# va troppo in swap
classifier = classifiers.NaiveBayesClassifier(training)

## Regressione con sklearn

In [54]:
# si può usare come label score che sono le stelle delle recensioni
# o su sentiment che è positive (4 5 stelle), neutral (3), negative (1, 2 stelle)
X_train, X_test, y_train, y_test = train_test_split(df['clean_text'], df['score'], random_state=0,
                                                            shuffle=True, train_size=0.80)

In [55]:
y_train.head()

8380     4
30491    2
28402    5
19373    1
30940    4
Name: score, dtype: int64

In [56]:
vect = CountVectorizer().fit(X_train)
X_train_vectorized = vect.transform(X_train)
X_train_vectorized.toarray()

model = LogisticRegression()
model.fit(X_train_vectorized, y_train)


predictions = model.predict(vect.transform(X_test))


In [57]:
predictions = model.predict(vect.transform(X_test))
precision, recall, fscore, support = score(y_test, predictions)
accuracy = sklearn.metrics.accuracy_score(y_test, predictions, normalize=True, sample_weight=None)

In [58]:
conf_mat_a = sklearn.metrics.confusion_matrix(y_test, predictions)
conf_mat_a

array([[ 343,   56,   47,   33,  164],
       [  78,   79,   59,   56,  127],
       [  44,   42,  144,  107,  260],
       [  18,   19,   73,  256,  627],
       [  52,   30,   52,  221, 4048]])

In [62]:
accuracy

0.6922530206112296